# 00 — API Smoke Tests (Day 1)

This notebook performs **minimal calls** to each API to confirm:
- network connectivity
- your API keys work (TomTom + TfL)
- response shapes are what we expect

**Do not hardcode keys here.** Put them in `.env` at the repo root.


In [1]:
from pathlib import Path
import sys

here = Path().resolve()
if (here / "src").exists():
    repo_root = here
elif (here.parent / "src").exists():
    repo_root = here.parent
else:
    repo_root = here.parents[1]

sys.path.insert(0, str((repo_root / "src").resolve()))
print("Repo root:", repo_root)


Repo root: /Users/davidbazalduamendez/Documents/GitHub/Short-term-traffic-congestion-prediction-for-London-Final-Project


## 1) Load settings (from `.env` / environment)

Create `.env` at the repo root:

```bash
cp .env.example .env
```

Fill in:
- `TOMTOM_API_KEY`
- `TFL_APP_KEY`


In [5]:
from traffic_forecast.config import get_settings, SettingsError

try:
    settings = get_settings()
    print("Settings loaded successfully.")
except SettingsError as e:
    print("Settings error:")
    print(e)
    raise


Settings loaded successfully.


## 2) Create HTTP session + clients


In [6]:
from traffic_forecast.http import HttpClient, build_session
from traffic_forecast.clients.tomtom import TomTomClient
from traffic_forecast.clients.tfl import TflClient
from traffic_forecast.clients.dft import DftRoadTrafficClient

http = HttpClient(
    session=build_session(user_agent=settings.user_agent),
    timeout_seconds=settings.http_timeout_seconds,
)

tomtom = TomTomClient(api_key=settings.tomtom_api_key, http=http)
tfl = TflClient(app_key=settings.tfl_app_key, app_id=settings.tfl_app_id, http=http)
dft = DftRoadTrafficClient(http=http)

print("Clients created.")


Clients created.


## 3) TomTom smoke test (Flow Segment Data)

We use a central London coordinate (Trafalgar Square area). You can change it later.


In [ ]:
import json
from datetime import datetime, timezone

lat, lon = 51.5079, -0.1281  #central London 

tomtom_payload = tomtom.flow_segment_data(lat=lat, lon=lon, zoom=10)
tomtom_metrics = tomtom.parse_metrics(tomtom_payload)

print(tomtom_metrics)
print("\nTop-level keys:", list(tomtom_payload.keys()))


FlowSegmentMetrics(frc='FRC3', current_speed=13.0, free_flow_speed=13.0, current_travel_time=87.0, free_flow_travel_time=87.0, confidence=1.0, road_closure=False, congestion_index=0.0)

Top-level keys: ['flowSegmentData']


## 4) TfL smoke test (Roads + disruptions)


In [8]:
roads = tfl.list_roads()
print("Type:", type(roads))

if isinstance(roads, list) and roads:
    print("First road keys:", list(roads[0].keys()) if isinstance(roads[0], dict) else type(roads[0]))
    road_id = roads[0].get("id") if isinstance(roads[0], dict) else None
    print("Using road_id:", road_id)
    if road_id:
        disruptions = tfl.road_disruptions(str(road_id))
        print("Disruptions type:", type(disruptions))
        if isinstance(disruptions, list) and disruptions:
            print("First disruption keys:", list(disruptions[0].keys()) if isinstance(disruptions[0], dict) else type(disruptions[0]))
        else:
            print("No disruptions returned (this can be normal).")
else:
    print("Unexpected roads response shape.")


Type: <class 'list'>
First road keys: ['$type', 'id', 'displayName', 'statusSeverity', 'statusSeverityDescription', 'bounds', 'envelope', 'url']
Using road_id: a1
Disruptions type: <class 'list'>
First disruption keys: ['$type', 'id', 'url', 'point', 'severity', 'ordinal', 'category', 'subCategory', 'comments', 'currentUpdate', 'currentUpdateDateTime', 'corridorIds', 'startDateTime', 'endDateTime', 'lastModifiedTime', 'levelOfInterest', 'location', 'status', 'geography', 'geometry', 'streets', 'isProvisional', 'hasClosures', 'roadDisruptionLines', 'roadDisruptionImpactAreas', 'recurringSchedules']


## 5) DfT smoke test (open API)


In [9]:
local_auth = dft.list_local_authorities(page_size=5, page_number=1)
count_points = dft.list_count_points(page_size=5, page_number=1)

print("Local authorities keys:", list(local_auth.keys()) if isinstance(local_auth, dict) else type(local_auth))
print("Count points keys:", list(count_points.keys()) if isinstance(count_points, dict) else type(count_points))


Local authorities keys: <class 'list'>
Count points keys: ['current_page', 'data', 'first_page_url', 'from', 'last_page', 'last_page_url', 'links', 'next_page_url', 'path', 'per_page', 'prev_page_url', 'to', 'total']


## 6) Save sample responses (no secrets)

This helps you keep stable examples in the repo **without** embedding keys.


In [10]:
from pathlib import Path

out_dir = repo_root / "sample_responses"
out_dir.mkdir(parents=True, exist_ok=True)

now = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")

(out_dir / f"tomtom_flow_segment_{now}.json").write_text(json.dumps(tomtom_payload, indent=2), encoding="utf-8")
(out_dir / f"tfl_roads_{now}.json").write_text(json.dumps(roads, indent=2), encoding="utf-8")
(out_dir / f"dft_local_authorities_{now}.json").write_text(json.dumps(local_auth, indent=2), encoding="utf-8")
(out_dir / f"dft_count_points_{now}.json").write_text(json.dumps(count_points, indent=2), encoding="utf-8")

print("Saved JSON files to:", out_dir)


Saved JSON files to: /Users/davidbazalduamendez/Documents/GitHub/Short-term-traffic-congestion-prediction-for-London-Final-Project/sample_responses
